In [210]:
import tensorflow as tf
print(tf.constant('Hellow from TensorFlow ' + tf.__version__ ))

tf.Tensor(b'Hellow from TensorFlow 2.1.0', shape=(), dtype=string)


In [211]:
import pandas as pd

# Load data
df = pd.read_csv("fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Transform label categories
ohe = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16)
ohe.fit(df["label"].to_numpy().reshape(-1,1))
print(ohe.categories_)
transformed = ohe.transform(df["label"].to_numpy().reshape(-1,1))

# Define X and y
X = df.drop("label", axis=1)
y = pd.DataFrame(data=transformed, columns=ohe.categories_[0])

# Delete variables that are no longer needed
# Nice to free up memory considering bigger datasets can take up quite a bit of memory
# and de-clutter the variable explorerer
del transformed
del df
del ohe

# Print shapes to validate
print(y.shape)
print(X.shape)

[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)]
(60000, 10)
(60000, 784)


In [213]:
y.head()

,0,1,2,3,4,5,6,7,8,9
0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0


In [214]:
X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,5,0,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,1,2,0,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
# Since the dataset contains greyscale pixel values in the range 0-255
# Rescale data to values between 0.01 and 1
# Avoid having 0 as input
X = X.apply(lambda x: (x * (0.99/255)) + 0.01, axis=1)
X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.01,0.01,0.01,0.010000,0.010000,0.01,0.01,0.010000,0.01,0.01,...,0.010000,0.01,0.01,0.010000,0.010000,0.010000,0.01,0.01,0.01,0.01
1,0.01,0.01,0.01,0.010000,0.010000,0.01,0.01,0.010000,0.01,0.01,...,0.010000,0.01,0.01,0.010000,0.010000,0.010000,0.01,0.01,0.01,0.01
2,0.01,0.01,0.01,0.010000,0.010000,0.01,0.01,0.029412,0.01,0.01,...,0.010000,0.01,0.01,0.126471,0.176941,0.010000,0.01,0.01,0.01,0.01
3,0.01,0.01,0.01,0.013882,0.017765,0.01,0.01,0.010000,0.01,0.01,...,0.021647,0.01,0.01,0.010000,0.010000,0.013882,0.01,0.01,0.01,0.01
4,0.01,0.01,0.01,0.010000,0.010000,0.01,0.01,0.010000,0.01,0.01,...,0.010000,0.01,0.01,0.010000,0.010000,0.010000,0.01,0.01,0.01,0.01


In [216]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, AveragePooling2D, Dropout, BatchNormalization, Reshape
from keras.optimizers import Adam

In [224]:
def fit_model(train_x, train_y, nodes, lrate, epochs, batch_size):

    # input_shape = (28, 28, 1)

    model = Sequential()

    # model.add(Reshape(input_shape, input_shape=input_shape))
    model.add(Conv2D(25, (5, 5), strides=1, input_shape=(28, 28, 1), activation="relu", name="Conv2d_1"))
    model.add(BatchNormalization(axis=3, name="Normalize_1"))
    model.add(Activation("relu"))

    model.add(MaxPooling2D((2, 2), name="Pooling2d_1"))
    model.add(Conv2D(50, (3, 3), strides=1, activation="relu", name="Conv2d_2"))
    model.add(AveragePooling2D((3, 3), name="AveragePooling"))
    model.add(Flatten())

    model.add(Dense(nodes, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation="softmax"))

    print(model.summary())


    model.compile(
        loss="categorical_crossentropy",
        #loss="sparse_categorical_crossentropy", # THIS DOES THE ONEHOTNENCODING INTERNALLY
        optimizer="adam",
        #optimizer=Adam(learning_rate=lrate),
        metrics=["accuracy"])

    model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size)
    return model

In [218]:
from sklearn.model_selection import KFold

In [219]:
X = X.to_numpy().reshape(-1, 28, 28, 1)

In [252]:
%%time

nodes = 200
learning_rate = 0.05
epochs = 100
batch_size=250

network = fit_model(X, y, nodes, learning_rate, epochs, batch_size)

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2d_1 (Conv2D)            (None, 24, 24, 25)        650       
_________________________________________________________________
Normalize_1 (BatchNormalizat (None, 24, 24, 25)        100       
_________________________________________________________________
activation_33 (Activation)   (None, 24, 24, 25)        0         
_________________________________________________________________
Pooling2d_1 (MaxPooling2D)   (None, 12, 12, 25)        0         
_________________________________________________________________
Conv2d_2 (Conv2D)            (None, 10, 10, 50)        11300     
_________________________________________________________________
AveragePooling (AveragePooli (None, 3, 3, 50)          0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 450)             

In [228]:
# predict with the fit model
def model_predict(model, testdata):
    Prediction = model.predict(testdata, verbose=0)
    return Prediction

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [247]:
test = pd.read_csv("fashion-mnist_test.csv")
test_y = test["label"].to_numpy().reshape(-1,1)

ohe = OneHotEncoder(categories="auto", sparse=False, dtype=np.int16)
ohe.fit(test_y)
print(ohe.categories_)
test_y = ohe.transform(test_y)

test_X = test.drop("label", axis=1)
test_X = test_X.to_numpy().reshape(-1, 28, 28, 1)
del test

[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)]


In [253]:
%%time

prediction = model_predict(network, test_X)

PredClass = list()
ActualClass = list()
for row in range(len(test_y)):
    index_maxscore1 = max(range(len(test_y[row])), key=test_y[row].__getitem__)
    ActualClass.append(index_maxscore1)
    index_maxscore2 = max(range(len(prediction[row])), key=prediction[row].__getitem__)
    PredClass.append(index_maxscore2)
    # print('Expected=%d, Got=%d' % (index_maxscore1, index_maxscore2))


accuracy = accuracy_metric(PredClass, ActualClass)
print("Accuracy: " + str(accuracy) + "%")

Accuracy: 64.07000000000001%
Wall time: 557 ms


100 Nodes network accuracy: 60.540000000000006%

200 Nodes network accuracy: 64.07000000000001%